# ベンチマークデータセットのGCN

In [17]:
import os
import networkx as nx
import torch
import numpy as np
import pandas as pd
import sys
sys.path.append("../codes")

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import TUDataset

from model import GCN
from train_test import train, test

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [29]:
# data load and split
dataset = TUDataset(root='data/TUDataset', name='MUTAG')

torch.manual_seed(12345)
dataset = dataset.shuffle()

train_index = int(0.7 * len(dataset))

train_dataset = dataset[:train_index]
test_dataset = dataset[train_index:]

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

In [30]:
# set model and train
model = GCN(hidden_channels=64, dataset=dataset)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

train_accs = []
test_accs = []


for epoch in range(1, 20):
    train(model, train_loader, criterion, optimizer, device)
    train_acc = test(model, train_loader, device)
    test_acc = test(model, test_loader, device)
    
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.6565, Test Acc: 0.6842
Epoch: 002, Train Acc: 0.6565, Test Acc: 0.6842
Epoch: 003, Train Acc: 0.6565, Test Acc: 0.6842
Epoch: 004, Train Acc: 0.6565, Test Acc: 0.6842
Epoch: 005, Train Acc: 0.6565, Test Acc: 0.6842
Epoch: 006, Train Acc: 0.6565, Test Acc: 0.6842
Epoch: 007, Train Acc: 0.6565, Test Acc: 0.6842
Epoch: 008, Train Acc: 0.6565, Test Acc: 0.6842
Epoch: 009, Train Acc: 0.6565, Test Acc: 0.6842
Epoch: 010, Train Acc: 0.6718, Test Acc: 0.7018
Epoch: 011, Train Acc: 0.7405, Test Acc: 0.7368
Epoch: 012, Train Acc: 0.7176, Test Acc: 0.7193
Epoch: 013, Train Acc: 0.7481, Test Acc: 0.7719
Epoch: 014, Train Acc: 0.7328, Test Acc: 0.7368
Epoch: 015, Train Acc: 0.7328, Test Acc: 0.7193
Epoch: 016, Train Acc: 0.7252, Test Acc: 0.7544
Epoch: 017, Train Acc: 0.7252, Test Acc: 0.7544
Epoch: 018, Train Acc: 0.7328, Test Acc: 0.7368
Epoch: 019, Train Acc: 0.7405, Test Acc: 0.7368


In [31]:
# log を保存する
save_dir = "data/TUDataset/MUTAG/result"
"""log の保存"""
# 保存先ディレクトリの作成
print(save_dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
# model(重み) の保存
torch.save(model.to("cpu").state_dict(), save_dir + "/model.pth")
# 学習結果の保存

df = pd.DataFrame()
df["train_acc"] = train_accs
df["test_acc"] = test_accs
df.to_csv(save_dir + "/log.csv")
# 学習曲線の保存
plt.style.use('ggplot')
plt.plot(train_accs, label="train")
plt.plot(test_accs, label="test")
plt.ylim(0,1)
plt.legend()
plt.savefig(save_dir + "/learning.png")
plt.clf()
plt.close()

data/TUDataset/MUTAG/result
